In [14]:
# @title Preamble
import numpy as np
import plotly.graph_objects as go

### Evaluation

In [15]:
# Parameters
alpha = 10
beta = 0.05
t_peak = 28
gamma = 0.5
S0 = 50
m0 = 10
c0 = 0
T = 50  # Total time to simulate
dt = 0.01  # Time step

# Initialize variables
S = 1200
m = 0
c = 0
time = 0

# Arrays to store the results for plotting
S_values = [S]
m_values = [m]
c_values = [c]
time_values = [time]

In [16]:
# @title Iterative solution
# Using Euler's method
while time < T:
    dS_dt = alpha * (np.log(1+m) / np.log(2.0)) - beta * (time - t_peak)**2 - gamma * c
    dm_dt = -time**2 - np.exp(c)
    dc_dt = np.log(time + 27)
    
    S += dS_dt * dt
    m += dm_dt * dt
    c += dc_dt * dt
    time += dt
    
    S_values.append(S)
    m_values.append(m)
    c_values.append(c)
    time_values.append(time)

C:\Users\joset\AppData\Local\Temp\ipykernel_14860\1449348193.py:4: RuntimeWarning:

invalid value encountered in log



### Plotting

In [17]:
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=time_values, y=S_values, mode='lines', marker = {'color' : 'blue'}, name="S(a,m,c)"),
    go.Scatter(x=time_values, y=m_values, mode='lines', marker = {'color' : 'red'}, name="m(t)"),
    go.Scatter(x=time_values, y=c_values, mode='lines', marker = {'color' : 'magenta'}, name="c(t)")
])
fig.update_layout(
    # title_text=title,
    height=1080*0.5,
    width=1920*0.6,
    xaxis_title="Time",
    yaxis_title="y"
)
fig.show()